# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os
import cv2

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

Using TensorFlow backend.


In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 10 #experiment with the batch size
#len(train_doc)//batch_size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [4]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = np.round(np.linspace(0,29,30)).astype(int) #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(train_doc)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,30,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image_resized=imresize(image,(100,100,3))
                    
                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255  #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
                batch_size = len(folder_list) - (batch_size*num_batches)
                batch_data = np.zeros((batch_size,30,100,100,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
                batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
                for folder in range(batch_size): # iterate over the batch_size
                    imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                    for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                        image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                        #crop the images and resize them. Note that the images are of 2 different shape 
                        #and the conv3D will throw error if the inputs in a batch have different shapes
                        image_resized=imresize(image,(100,100,3))
                    
                        batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255 #normalise and feed in the image
                        batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255  #normalise and feed in the image
                        batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255 #normalise and feed in the image
                    
                    batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

## Changing the code a bit for modularity ####
### Creating a custom Data  Generator class ####

In [5]:
class DataGenerator:
    def __init__(self, width=120, height=120, frames=30, channel=3, 
                 crop = True, normalize = False, affine = False, flip = False):
        self.width = width   # X dimension of the image
        self.height = height # Y dimesnion of the image
        self.frames = frames # length/depth of the video frames
        self.channel = channel # number of channels in images 3 for color(RGB) and 1 for Gray  
        self.affine = affine # augment data with affine transform of the image
        self.flip = flip
        self.normalize =  normalize
        self.crop = crop

    # Helper function to generate a random affine transform on the image
    def __get_random_affine(self): # private method
        dx, dy = np.random.randint(-1.7, 1.8, 2)
        M = np.float32([[1, 0, dx], [0, 1, dy]])
        return M

    # Helper function to initialize all the batch image data and labels
    def __init_batch_data(self, batch_size): # private method
        batch_data = np.zeros((batch_size, self.frames, self.width, self.height, self.channel)) 
        batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
        return batch_data, batch_labels

    def __load_batch_images(self, source_path, folder_list, batch_num, batch_size, t): # private method
    
        batch_data,batch_labels = self.__init_batch_data(batch_size)
        # We will also build a agumented batch data
        if self.affine:
            batch_data_aug,batch_labels_aug = self.__init_batch_data(batch_size)
        if self.flip:
            batch_data_flip,batch_labels_flip = self.__init_batch_data(batch_size)

        #create a list of image numbers you want to use for a particular video
        img_idx = [x for x in range(0, self.frames)] 

        for folder in range(batch_size): # iterate over the batch_size
            # read all the images in the folder
            imgs = sorted(os.listdir(source_path+'/'+ t[folder + (batch_num*batch_size)].split(';')[0])) 
            # Generate a random affine to be used in image transformation for buidling agumented data set
            M = self.__get_random_affine()
            
            #  Iterate over the frames/images of a folder to read them in
            for idx, item in enumerate(img_idx): 
                image = cv2.imread(source_path+'/'+ t[folder + (batch_num*batch_size)].strip().split(';')[0]+'/'+imgs[item], cv2.IMREAD_COLOR)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
                #crop the images and resize them. Note that the images are of 2 different shape 
                #and the conv3D will throw error if the inputs in a batch have different shapes  
                if self.crop:
                    image = self.__crop(image)
                # If normalize is set normalize the image else use the raw image.
                if self.normalize:
                    resized = self.__normalize(self.__resize(image))
                else:
                    resized = self.__resize(image)
                
                batch_data[folder,idx] = resized
                if self.affine:
                    batch_data_aug[folder,idx] = self.__affine(resized, M)   
                if self.flip:
                    batch_data_flip[folder,idx] = self.__flip(resized)   

            batch_labels[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
            
            if self.affine:
                batch_labels_aug[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
            
            if self.flip:
                if int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==0:
                    batch_labels_flip[folder, 1] = 1
                elif int(t[folder + (batch_num*batch_size)].strip().split(';')[2])==1:
                    batch_labels_flip[folder, 0] = 1
                else:
                    batch_labels_flip[folder, int(t[folder + (batch_num*batch_size)].strip().split(';')[2])] = 1
        
        if self.affine:
            batch_data = np.append(batch_data, batch_data_aug, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_aug, axis = 0) 
        if self.flip:
            batch_data = np.append(batch_data, batch_data_flip, axis = 0) 
            batch_labels = np.append(batch_labels, batch_labels_flip, axis = 0) 

        return batch_data, batch_labels
    
    def generator(self, source_path, folder_list, batch_size): # public method
        print( 'Source path = ', source_path, '; batch size =', batch_size)
        while True:
            t = np.random.permutation(folder_list)
            num_batches = len(folder_list)//batch_size # calculate the number of batches
            for batch in range(num_batches): # we iterate over the number of batches
                # you yield the batch_data and the batch_labels, remember what does yield do
                yield self.__load_batch_images(source_path, folder_list, batch, batch_size, t) 
            
            # Code for the remaining data points which are left after full batches
            if (len(folder_list) != batch_size*num_batches):
                batch_size = len(folder_list) - (batch_size*num_batches)
                yield self.__load_batch_images(source_path, folder_list, num_batches, batch_size, t)

    # Helper function to perfom affice transform on the image
    def __affine(self, image, M):
        return cv2.warpAffine(image, M, (image.shape[0], image.shape[1]))

    # Helper function to flip the image
    def __flip(self, image):
        return np.flip(image,1)
    
    # Helper function to normalise the data
    def __normalize(self, image):
        return image/127.5-1
    
    # Helper function to resize the image
    def __resize(self, image):
        return cv2.resize(image, (self.width,self.height), interpolation = cv2.INTER_AREA)
    
    # Helper function to crop the image
    def __crop(self, image):
        if image.shape[0] != image.shape[1]:
            return image[0:120, 20:140]
        else:
            return image

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
#train_path = '/notebooks/storage/Final_data/Collated_training/train'
#val_path = '/notebooks/storage/Final_data/Collated_training/val'
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

nb_classes = 5
input_shape = (30,120,120,3)
#write your model here
model = Sequential()
model.add(Conv3D( 8, (3,3,3), activation='relu', input_shape=input_shape ))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model.add(Conv3D(16, (3,3,3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model.add(Conv3D(32, (3,3,3), activation='relu'))
model.add(Conv3D(32, (3,3,3), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))
model.add(Conv3D(64, (2,2,2), activation='relu'))
model.add(Conv3D(64, (2,2,2), activation='relu'))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2)))

model.add(Flatten())
model.add(Dense(512))
model.add(Dropout(0.5))
model.add(Dense(256))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [8]:
optimiser = optimizers.Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_1 (Conv3D)            (None, 28, 118, 118, 8)   656       
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 28, 59, 59, 8)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 26, 57, 57, 16)    3472      
_________________________________________________________________
max_pooling3d_2 (MaxPooling3 (None, 26, 28, 28, 16)    0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 24, 26, 26, 32)    13856     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 22, 24, 24, 32)    27680     
_________________________________________________________________
max_pooling3d_3 (MaxPooling3 (None, 22, 12, 12, 32)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [9]:
train_gen = DataGenerator()
val_gen = DataGenerator()

train_generator = train_gen.generator(train_path, train_doc, batch_size)
val_generator = train_gen.generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1
print(validation_steps)

10


Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [12]:
#model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
#                    callbacks=callbacks_list, validation_data=val_generator, 
#                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=1, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator,validation_steps=3,workers=1)

Source path =  Project_data/val ; batch size = 50
Source path =  Project_data/train ; batch size = 50
Epoch 1/1
14/14 [==============================] - 111s 8s/step - loss: 12.6300 - categorical_accuracy: 0.2116 - val_loss: 12.1423 - val_categorical_accuracy: 0.2467

Epoch 00001: saving model to model_init_2020-09-1316_04_03.634826/model-00001-12.70639-0.20664-12.14230-0.24667.h5


In [14]:
K.clear_session()

##### Since the pathway is built, lets try out with different models

# 3D CNN's #

## Model 1A ##

In [15]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

nb_classes = 5
input_shape = (30,120,120,3)

## Define the model ##
model = Sequential()

# 1st layer group
model.add(Conv3D(16, 3, 3, 3, activation='relu',padding='same', name='conv1',subsample=(1, 1, 1),input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),border_mode='valid', name='pool1'))

# 2nd layer group
model.add(Conv3D(32, 3, 3, 3, activation='relu',padding='same', name='conv2',subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),border_mode='valid', name='pool2'))

# 3rd layer group
model.add(Conv3D(64, 3, 3, 3, activation='relu',padding='same', name='conv3a',subsample=(1, 1, 1)))
model.add(Conv3D(64, 3, 3, 3, activation='relu',padding='same', name='conv3b',subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),border_mode='valid', name='pool3'))

# 4th layer group
model.add(Conv3D(128, 3, 3, 3, activation='relu',padding='same', name='conv4a',subsample=(1, 1, 1)))
model.add(Conv3D(128, 3, 3, 3, activation='relu',padding='same', name='conv4b',subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),border_mode='valid', name='pool4'))

# 5th layer group
model.add(Conv3D(256, 3, 3, 3, activation='relu',padding='same', name='conv5a',subsample=(1, 1, 1)))
model.add(Conv3D(256, 3, 3, 3, activation='relu',padding='same', name='conv5b',subsample=(1, 1, 1)))
model.add(ZeroPadding3D(padding=(0, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),padding='valid', name='pool5'))
model.add(Flatten())

# FC layers group
model.add(Dense(512, activation='relu', name='fc6'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', name='fc7'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

#Compile your model
optimiser = optimizers.Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.summary()


## Part A ##
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                   validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1", input_shape=(30, 120, ..., strides=(1, 1, 1))`
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (3, 3, 3), activation="relu", padding="same", name="conv2", strides=(1, 1, 1))`
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid")`
/mnt/

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 15, 30, 30, 32)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 15, 30, 30, 64)    55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 15, 30, 30, 64)    110656    
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 7, 15, 15, 64)     0         
__________

In [12]:
K.clear_session()

In [12]:
! nvidia-smi

Mon Sep 14 05:37:12 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 390.30                 Driver Version: 390.30                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P0    79W / 149W |      0MiB / 11441MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [12]:
from numba import cuda 
device = cuda.get_current_device()
device.reset()

## Model 1B ##

In [13]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

nb_classes = 5
input_shape = (30,120,120,3)

## Define the model ##
model = Sequential()

# 1st layer group
model.add(Conv3D(16, 3, 3, 3, activation='relu',padding='same', name='conv1',subsample=(1, 1, 1),input_shape=input_shape))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),border_mode='valid', name='pool1'))

# 2nd layer group
model.add(Conv3D(32, 3, 3, 3, activation='relu',padding='same', name='conv2',subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),border_mode='valid', name='pool2'))

# 3rd layer group
model.add(Conv3D(64, 3, 3, 3, activation='relu',padding='same', name='conv3a',subsample=(1, 1, 1)))
model.add(Conv3D(64, 3, 3, 3, activation='relu',padding='same', name='conv3b',subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),border_mode='valid', name='pool3'))

# 4th layer group
model.add(Conv3D(128, 3, 3, 3, activation='relu',padding='same', name='conv4a',subsample=(1, 1, 1)))
model.add(Conv3D(128, 3, 3, 3, activation='relu',padding='same', name='conv4b',subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),border_mode='valid', name='pool4'))

# 5th layer group
model.add(Conv3D(256, 3, 3, 3, activation='relu',padding='same', name='conv5a',subsample=(1, 1, 1)))
model.add(Conv3D(256, 3, 3, 3, activation='relu',padding='same', name='conv5b',subsample=(1, 1, 1)))
model.add(ZeroPadding3D(padding=(0, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),padding='valid', name='pool5'))
model.add(Flatten())

# FC layers group
model.add(Dense(512, activation='relu', name='fc6'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', name='fc7'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

#Compile your model
optimiser = optimizers.Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.summary()

train_gen_mod = DataGenerator(normalize=True,affine=True,flip=True)
val_gen_mod = DataGenerator(normalize=True,affine=True,flip=True)

train_generator_mod = train_gen_mod.generator(train_path, train_doc, batch_size)
val_generator_mod = val_gen_mod.generator(val_path, val_doc, batch_size)


## Part A ##
model.fit_generator(train_generator_mod, steps_per_epoch=100, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator_mod, 
                   validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1", input_shape=(30, 120, ..., strides=(1, 1, 1))`
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2), name="pool1", padding="valid")`
  from ipykernel import kernelapp as app
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(32, (3, 3, 3), activation="relu", padding="same", name="conv2", strides=(1, 1, 1))`
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), name="pool2", padding="valid")`
/mnt/

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv3D)               (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2 (Conv3D)               (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
pool2 (MaxPooling3D)         (None, 15, 30, 30, 32)    0         
_________________________________________________________________
conv3a (Conv3D)              (None, 15, 30, 30, 64)    55360     
_________________________________________________________________
conv3b (Conv3D)              (None, 15, 30, 30, 64)    110656    
_________________________________________________________________
pool3 (MaxPooling3D)         (None, 7, 15, 15, 64)     0         
__________

## Model 2

In [21]:
from keras.models import Sequential, Model
from keras.layers import Dense, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, ZeroPadding3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

nb_classes = 5
input_shape = (30,120,120,3)

## Define the model ##
model = Sequential()

# 1st layer group
model.add(Conv3D(16, 3, 3, 3, activation='relu',padding='same', name='conv1a',subsample=(1, 1, 1),input_shape=input_shape))
#(adding few more conv3D)
model.add(Conv3D(16, 3, 3, 3, activation='relu',padding='same', name='conv1b',subsample=(1, 1, 1)))
model.add(Conv3D(16, 3, 3, 3, activation='relu',padding='same', name='conv1c',subsample=(1, 1, 1)))
model.add(Conv3D(16, 3, 3, 3, activation='relu',padding='same', name='conv1d',subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(1, 2, 2), strides=(1, 2, 2),border_mode='valid', name='pool1'))

# 2nd layer group
model.add(Conv3D(32, 3, 3, 3, activation='relu',padding='same', name='conv2a',subsample=(1, 1, 1)))
model.add(Conv3D(32, 3, 3, 3, activation='relu',padding='same', name='conv2b',subsample=(1, 1, 1)))
model.add(Conv3D(32, 3, 3, 3, activation='relu',padding='same', name='conv2c',subsample=(1, 1, 1)))
model.add(Conv3D(32, 3, 3, 3, activation='relu',padding='same', name='conv2d',subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),border_mode='valid', name='pool2'))

# 3rd layer group
model.add(Conv3D(64, 3, 3, 3, activation='relu',padding='same', name='conv3a',subsample=(1, 1, 1)))
model.add(Conv3D(64, 3, 3, 3, activation='relu',padding='same', name='conv3b',subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),border_mode='valid', name='pool3'))

# 4th layer group
model.add(Conv3D(128, 3, 3, 3, activation='relu',padding='same', name='conv4a',subsample=(1, 1, 1)))
model.add(Conv3D(128, 3, 3, 3, activation='relu',padding='same', name='conv4b',subsample=(1, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),border_mode='valid', name='pool4'))

# 5th layer group
model.add(Conv3D(256, 3, 3, 3, activation='relu',padding='same', name='conv5a',subsample=(1, 1, 1)))
model.add(Conv3D(256, 3, 3, 3, activation='relu',padding='same', name='conv5b',subsample=(1, 1, 1)))
model.add(ZeroPadding3D(padding=(0, 1, 1)))
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2),padding='valid', name='pool5'))
model.add(Flatten())

# FC layers group
model.add(Dense(512, activation='relu', name='fc6'))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu', name='fc7'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes, activation='softmax'))

#Compile your model
optimiser = optimizers.Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model.summary()

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                   validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1a", input_shape=(30, 120, ..., strides=(1, 1, 1))`
  
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1b", strides=(1, 1, 1))`
  app.launch_new_instance()
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1c", strides=(1, 1, 1))`
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(16, (3, 3, 3), activation="relu", padding="same", name="conv1d", strides=(1, 1, 1))`
/mnt/disks/use

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1a (Conv3D)              (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
conv1b (Conv3D)              (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
conv1c (Conv3D)              (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
conv1d (Conv3D)              (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
pool1 (MaxPooling3D)         (None, 30, 60, 60, 16)    0         
_________________________________________________________________
conv2a (Conv3D)              (None, 30, 60, 60, 32)    13856     
_________________________________________________________________
conv2b (Conv3D)              (None, 30, 60, 60, 32)    27680     
__________

# CNN+LSTM #

## Model 3A

In [25]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

nb_classes = 5
input_shape = (30,120,120,3)

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))


model.add(TimeDistributed(Flatten()))


model.add(GRU(64))
model.add(Dropout(0.25))

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_classes, activation='softmax'))

optimiser = optimizers.Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


model.summary()

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                   validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_40 (TimeDis (None, 30, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_41 (TimeDis (None, 30, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_42 (TimeDis (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_43 (TimeDis (None, 30, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_44 (TimeDis (None, 30, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_45 (TimeDis (None, 30, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_46 (TimeDis (None, 30, 30, 30, 64)    18496     
__________

## Model 3B

In [15]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

nb_classes = 5
input_shape = (30,120,120,3)

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),input_shape=input_shape))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))


model.add(TimeDistributed(Flatten()))


model.add(GRU(64))
model.add(Dropout(0.25))

model.add(Dense(512,activation='relu'))
model.add(Dropout(0.25))

model.add(Dense(nb_classes, activation='softmax'))

optimiser = optimizers.Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


model.summary()

model.fit_generator(train_generator_mod, steps_per_epoch=100, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator_mod, 
                   validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_7 (TimeDist (None, 30, 30, 30, 64)    18496     
__________

#### Look at the accuracy measure of the CNN+LSTM stack. This far better than any 3d CNN's ####

## Model 4 ##

In [13]:
## Lets use the Xception Net as CNN stack ##
from keras.applications.xception import Xception
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout,GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras
import tensorflow as tf


nb_classes = 5
input_shape = (30,120,120,3)

base_model = Xception(weights='imagenet', include_top=False)

model = Sequential()
model.add(TimeDistributed(base_model,input_shape=input_shape))

for layer in base_model.layers:
    layer.trainable = False

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))

#model.add(GRU(128,return_sequences=True))
#model.add(GRU(128,return_sequences=True))
model.add(GRU(128))

model.add(Dropout(0.25))
        
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.25))
        
model.add(Dense(nb_classes, activation='softmax'))

optimiser = optimizers.Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


model.summary()

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator, 
                   validation_steps=validation_steps, class_weight=None, workers=4, initial_epoch=0)

83689472/83683744 [==============================] - 6s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_1 (TimeDist (None, 30, 4, 4, 2048)    20861480  
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 4, 4, 2048)    8192      
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 2, 2, 2048)    0         
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 8192)          0         
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               3195264   
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (De

In [16]:
## Lets use the Xception Net as CNN stack ##
from keras.applications.xception import Xception
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation,Dropout,GlobalAveragePooling2D
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras
import tensorflow as tf


nb_classes = 5
input_shape = (30,120,120,3)

base_model = Xception(weights='imagenet', include_top=False)

model = Sequential()
model.add(TimeDistributed(base_model,input_shape=input_shape))

for layer in base_model.layers:
    layer.trainable = False

model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
model.add(TimeDistributed(Flatten()))

model.add(GRU(128,return_sequences=True))
model.add(GRU(128,return_sequences=True))
model.add(GRU(128))

model.add(Dropout(0.25))
        
model.add(Dense(32,activation='relu'))
model.add(Dropout(0.25))
        
model.add(Dense(nb_classes, activation='softmax'))

optimiser = optimizers.Adam() 
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])


model.summary()

model.fit_generator(train_generator_mod, steps_per_epoch=100, epochs=num_epochs, verbose=1, 
                  callbacks=callbacks_list, validation_data=val_generator_mod, 
                   validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

83689472/83683744 [==============================] - 6s 0us/step
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_14 (TimeDis (None, 30, 4, 4, 2048)    20861480  
_________________________________________________________________
time_distributed_15 (TimeDis (None, 30, 4, 4, 2048)    8192      
_________________________________________________________________
time_distributed_16 (TimeDis (None, 30, 2, 2, 2048)    0         
_________________________________________________________________
time_distributed_17 (TimeDis (None, 30, 8192)          0         
_________________________________________________________________
gru_2 (GRU)                  (None, 30, 128)           3195264   
_________________________________________________________________
gru_3 (GRU)                  (None, 30, 128)           98688     
_________________________________________________________________
gru_4 (GRU)